In [2]:
# -*- coding: utf-8 -*-
"""
Created on Mon Dec  3 22:59:59 2018

@author: maroc
"""

# -*- coding: utf-8 -*-
"""
Created on Mon Dec  3 22:59:59 2018

@author: maroc
"""
#preprocess imports
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
import numpy as np
import pandas as pd


df = pd.read_csv("concat2.csv", sep=',') #read source 1

suppl = Chem.SmilesMolSupplier('1000000.smi') #read source 2

#turn data from source to into pd.DataFrame
rows_list = []
for mol in suppl:
    if mol is None: continue
    name = mol.GetProp("_Name")
    dict1 = {'molecule':name, 'Smiles':mol}
    rows_list.append(dict1)
X = pd.DataFrame(rows_list)

#merge sources
df = df.merge(X, how='inner', on='molecule')

In [3]:
#preprocess data
df = df.dropna(axis=0, how='any') #remove nans
X = df[['Smiles']] #get x
y = df.loc[:, df.columns != 'Smiles']#remove non-numeric columns from targets
y = y.loc[:, y.columns != 'molecule']
res_col = y.columns.get_loc('QPlogS')#save position of target parameter 
#get bit vector for training 1st model 
X['Smiles'] = X['Smiles'].apply(lambda x:
    AllChem.GetMorganFingerprintAsBitVect(x, 2))

#remove rows with nonumeric entities in numeric columns
rows_with_strings  = y.apply(
       lambda row : 
          any([ isinstance(e, str) for e in row ])
       , axis=1)     
y = y[~rows_with_strings]   
X = X[~rows_with_strings] 

#transform data
X = X['Smiles'].values.tolist()
y = y.values.tolist()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [4]:
#split data to train/test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [5]:
#separate data to minibatches
num = 100
Xs = np.array_split(X_train, num)
ys = np.array_split(y_train, num)

Xs2 = np.array_split(X_test, num)
ys2 = np.array_split(y_test, num)


In [6]:
#training imports
from random import randint
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
#select alpha and initial parameters for the first model by 1% of data
regr = linear_model.MultiTaskLassoCV(n_alphas=150, max_iter=1000, cv=10, n_jobs = -1, verbose=True)

#train on some minibatch
i = randint(0, 99)
X_tr = Xs[i]
y_tr = ys[i]
X_te = Xs2[i]
y_te = ys2[i]
regr.fit(X_tr, y_tr)
#save parameters
alphas = regr.alphas_
alpha = alphas[len(alphas)-1]*20
coeffs = regr.coef_
ints = regr.intercept_ 
print("batch", i)
print(alpha)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
.......................................................................................

In [ ]:
#train first model on the whole dataset using parameters from previous step
regr_main = linear_model.MultiTaskLasso(alpha = alpha, max_iter=1000, warm_start=True )
regr_main.coef_ = coeffs
regr_main.intercept_ = ints
regr_main.fit(X_train, y_train)

In [ ]:
#check 1st model
pred = regr_main.predict(X_test)
err = ((y_test - pred)**2).mean(axis=0)

print(err) #mse on main target was about 0.7

In [ ]:
#get pre-initial alphas for second model
#auto-search gave some poor results
alph_num = 20
alphas =np.ones(alph_num,)*alphas[len(alphas)-1]*20
for i in range(alph_num-1):
    alphas[i+1] = 0.7*alphas[i]

In [ ]:
#select alpha and initial parameters for the second model by 1% of data
regr1 =make_pipeline(PolynomialFeatures(degree=2, interaction_only=False), linear_model.LassoCV( alphas=alphas, max_iter=5000, verbose=True, cv=20, n_jobs = -1))

i = randint(0, 99)
X_tr = Xs[i]
y_tr = ys[i]
X_te = Xs2[i]
y_te = ys2[i]
pred = regr_main.predict(X_tr) #get inputs by previous model
target = y_tr[:, res_col] #get target column
regr1.fit(pred, target)

In [ ]:
#check results of 2nd pre-model
pred2 = regr_main.predict(X_te)
pred_fin = regr1.predict(pred2)
err = (mean_squared_error(y_te[:, res_col], pred_fin))
print("iter", i)
print("err", err)

In [ ]:
#save parameters
coef = regr1.named_steps.lassocv.coef_
alpha = regr1.named_steps.lassocv.alpha_ 
inter = regr1.named_steps.lassocv.intercept_


In [ ]:
#initialize 2nd model with ready parameters
regr_fin = make_pipeline(PolynomialFeatures(degree=2, interaction_only=False), linear_model.Lasso( alpha=alpha, max_iter=5000,  warm_start=True))
regr_fin.coef_ = coef
regr_fin.intercept_ = ints
#get input for training 2-nd model of full dataset
pred = regr_main.predict(X_train)
#get input column for training
target = np.asarray(y_train)[:, res_col]
regr_fin.fit(pred, target) #train 2nd model

In [ ]:
#check full model on test set
input_fin = regr_main.predict(X_test)
res_fin = regr_fin.predict(input_fin)
print(mean_squared_error(np.asarray(y_test)[:, res_col], res_fin))
#mse is 0.5465438787065124
# in case of linear regresion it werec about 0.58

In [ ]:
#Validate final results
from sklearn.metrics import explained_variance_score

print(explained_variance_score(np.asarray(y_test)[:, res_col], res_fin))
#explained variance 0.7576613962521102

In [ ]:
#Save results
#create df for columns
y2 = df.loc[:, df.columns != 'Smiles']
y2 = y2.loc[:, y2.columns != 'molecule']
#we don't have actual parameters - only predictions from bits
#This is true for all parameters, but I noted it for purpose only
#as predicting QPlogS from QPlogS would sound a bit strange
y2 = y2.rename(columns={"QPlogS": "PredictionFromBits"})

In [ ]:
poly = regr_fin.named_steps.polynomialfeatures
fts = poly.get_feature_names(y2.columns)
results =pd.DataFrame(coef, fts)
results.to_csv("results.csv", sep=';')